In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy as sp

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# necessary import

# for visualization
import seaborn as sns
from matplotlib import pyplot as plt


# for dimension reduction
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

# ML algorithms
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_predict, train_test_split

from PIL import Image
from scipy import ndimage

# ML algorithms evaluations
from sklearn.metrics import confusion_matrix
print('setup complete')

save the file path in a variable to enable easy accessing

I am going to working with only the test set, ie using only the test set to build the model and see
if I can train a model comparable in accuracy to using the large training set. I will be usin the training set as the test set

That is:
* X_train = test_set.csv
* X_test = train_set.csv

In [ ]:
# the dataset have already been split into train, test set
train_set = "/kaggle/input/mnist-in-csv/mnist_test.csv"
test_set = "/kaggle/input/mnist-in-csv/mnist_train.csv"

# dump the train dataset into pandas dataframe for easy manipulation
# and plotting (don't touch the test set until when you are finally going to
# test in on the final model)

X_train = pd.read_csv(train_set, dtype=np.uint8)
X_test = pd.read_csv(test_set, dtype=np.uint8)
X_train.head()

# a quirk of mine, to make me immediately aware when the code execution end
# or if it has already been executed
print('setup complete') 

In [ ]:
X_train.shape, X_test.shape

In [ ]:
# to check out what we are going to be working with
X_train.info()

Ok, the train_set contains images instance of dimension 28X28
and it also have the target label in it, which we will have to remove it. And we can easily visualize any image using matplotlib

In [ ]:
# remove target label
y_train = X_train['label'].copy()
y_test = X_test['label'].copy()
X_train.drop('label', axis=1, inplace=True)
X_test.drop('label',axis=1, inplace=True)
print('setup complete')

In [ ]:
# you can change the random_image value to visualize any other image
random_image = 1000
some_digit = X_train.iloc[random_image] # select any number, change to select any number
plt.imshow(some_digit.values.reshape(28, 28))
print('label', y_train.iloc[random_image])

In [ ]:
# to check if the samples are evenly distributed

plt.bar(y_train.value_counts().index, y_train.value_counts())
# the digit image 5 has the lowest number of instances while image 1 has the highest
# but all within acceptable ranges, so Good to Go

* from the above histogram it is obvious than the number image 1 has the highest instances and number image 5 has the lowest, but the difference between them is not overwhelming and not skewed toward a class and can therefore be used in training a model

In [ ]:
# we will visualize the datasets to gain insights
tsne = TSNE(n_components=2)

shuffled_index = np.random.permutation(len(X_train))

X_train_tsne = tsne.fit_transform(X_train)
print('setup complete')

In [ ]:
plt.figure(figsize=(10, 8))
for i in range(10):
    # select only the indexes of each labeled examples
    index = y_train == i
    plt.scatter(X_train_tsne[index, 0], X_train_tsne[index, 1],
                 label=f"{i}")
plt.legend()
print('setup complete')

from the plot each class can be easily be differentiated and therefore we can build a model that can have high accuracy

And also from the plot we can gain a lot of insight and problems the ML algorithm will face, for examples

1. We can build a model that can predict perfectly without any error between 0 and 1

2. There is a lot of similarities between 4 and 9, 5 and 3 and 8, 1 and 2 and 7 (they get confused for one other, maybe as a result of bad handwriting)

3. Two (2) get confused as one(1) but not the other way round, also seven(7) also get confused as two (2) and vice versa

4. Only six (6) and zero (0) are mostly if not 99% correctly classified

We are going to set a base line here, that is the lowest accuracy our model can achieve, and after that we train a model that can beat that base line


We are going to set the base line using the raw dataset

In [ ]:
knn = KNeighborsClassifier(n_jobs=-1)
knn.fit(X_train, y_train)
print("Training score: ", knn.score(X_train, y_train))
print("Test score: ", knn.score(X_test, y_test))

In [ ]:
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
print("Training score: ", log_reg.score(X_train, y_train))
print("Test score: ", log_reg.score(X_test, y_test))

In [ ]:
svc = LinearSVC()
svc.fit(X_train, y_train)
print("Training score: ", svc.score(X_train, y_train))
print("Test score: ", svc.score(X_test, y_test))

In [ ]:
predictions = cross_val_predict(log_reg, X_train, y_train, cv=3)

In [ ]:
conf_mx = confusion_matrix(y_train, predictions)
row_sum = conf_mx.sum(axis=1, keepdims=True)
norm_conf_mx = conf_mx / row_sum
np.fill_diagonal(norm_conf_mx, 0)
plt.matshow(norm_conf_mx, cmap=plt.cm.gray)
plt.xlabel('Predicted Class')
plt.ylabel('True Class')

**from the above plot we can clearly see how our model is performing and the errors it is making in predicting the various class, but as you will come to see, some of the errors it is making in classifying the labels are errors even we humans would tend to make when told to classify them (due to the way the number was written)**

from the above diagram we can make several observation about our model, or rather confirmed the suspicions we had about some image when we visualize it using TSNE 

1. all number zero (0) was classified correctly except some very little instances were it was confused as number six (6)

2. all number one (1) was also classified correctly except some confusion were it was classified as 7

3. number two (2) sometimes get confused as 0, 1 and 7

4. number three (3) sometimes get confused as 2 and 5

5. number four(4) get a little confused as 2 and mostly mistaken as 9
so on.... you get the flow here right?

But I am going to focus on the two brightest spot that is at the point where number 4 is getting mistaken as 9 a lot, and number (8) is getting mistaken as 5 (how is that possible?)

In [ ]:
conf_mx

In [ ]:
# first let see why 4 is mistaken as 9
a, b = 4, 9
true_positive = X_train[(y_train == a) & (predictions == a)]

# what we really need to look at

# number 4 mistaken as 9
false_negative = X_train[(y_train == a) & (predictions == b)]
# number 9 mistaken as 4
false_positive = X_train[(y_train == b) & (predictions == a)]

# don't need this
true_negative = X_train[(y_train == b) & (predictions == b)]

In [ ]:
def plot_mistaken_identities(array):
    dim = int(np.sqrt(len(array))) + 1
    fig, axes = plt.subplots(dim, dim)
    for axe, img in zip(axes.ravel(), array.values):
        axe.imshow(img.reshape(28, 28))

In [ ]:
plot_mistaken_identities(false_negative.iloc[:15])
# from the below plot the number 4 that is wrongly classified as 9 are
# those that actually looks like number 9! and are something if we humans
# will have errors in classifying (tag someone who has handwriting that
# makes reading their books a challenge, thank God for word processing SW)

In [ ]:
plot_mistaken_identities(false_positive.iloc[30:])
# Jesus! some people need to go back to creche school to relearn how to 
# write numbers, or maybe this number were written by creche students who 
# hasn't yet mastered how to write

# because I see no reason why a 9 should look as exactly as 4, then how
# would your 4 look like then. YOU ARE BREAKING MY MODEL!! and making it look
# dumb

Play around and try to see other errors the model made, (which on my part the blame should be on the model but the writers) and you will see strong reasons why a good handwritting foundation is important in modern life.

But how can we get around problems that the model encountered and try to increase our model predictive accuracy?

1. We can try a more complex ML algorithm like Random Forest, SVC, deep learning etc.

2. rotate the image and add it to your data set (called data augmentation)

3. use any image processing module (eg Pillow, OpenCV) to sharpen, extract edges or any filter you desire to extract features from your image (this should be begin your path to computer vision)

**What ever path you take, try to increase it accuracy to above 98%. GoodLuck**


In [ ]:
def rotate_image(image, deg=36):
    """Rotate a given image as specified by the degree"""
    # reshape the array to the image dimension
    # rotate the image
    image_rotated = sp.ndimage.rotate(image, deg)
    return image_rotated

def shift_image(image, stride=4, direction='up'):
    """shift the image 4 pixel in the specified direction"""
    if direction == 'up':
        shifted = sp.ndimage.interpolation.shift(image, [-1, 0])
    elif direction == 'down':
        shifted = sp.ndimage.interpolation.shift(image, [1, 0])
    elif direction == 'left':
        shifted = sp.ndimage.interpolation.shift(image, [0, -1])
    elif direction == 'right':
        shifted = sp.ndimage.interpolation.shift(image, [0, 1])
    return list(shifted.ravel())

In [ ]:
def create_rotated_shifted_images(dataframe):
    # creating data containers
    image_36 = {direction: [] for direction in ['left', 'right', 'up', 'down']}
    
    image_72 = {direction: [] for direction in ['left', 'right', 'up', 'down']}
    
    image_108 = {direction: [] for direction in ['left', 'right', 'up', 'down']}
    
    image_144 = {direction: [] for direction in ['left', 'right', 'up', 'down']}
    
    image_180 = {direction: [] for direction in ['left', 'right', 'up', 'down']}
    
    image_216 = {direction: [] for direction in ['left', 'right', 'up', 'down']}
    
    image_252 = {direction: [] for direction in ['left', 'right', 'up', 'down']}
    
    image_288 = {direction: [] for direction in ['left', 'right', 'up', 'down']}
    
    image_324 = {direction: [] for direction in ['left', 'right', 'up', 'down']}
    
    image_360 = {direction: [] for direction in ['left', 'right', 'up', 'down']}
    
    container = [image_36, image_72, image_108, image_144, image_180, image_216, image_252, image_288, image_324, image_360]
    rotations = [36, 72, 108, 144, 180, 216, 252, 288, 324, 360]
    for array_index in dataframe.index:
        image = dataframe.iloc[array_index].values.reshape(28, 28)
        count = 0
        for deg, deg_container in zip(rotations, container):
            # rotate the image
            rotated_image = rotate_image(image, deg)
            # resize the image of any rotated image that got larger than the (28, 28) dimension
            if rotated_image.shape[0] != 28:
                distorted_image = Image.fromarray(rotated_image)
                restored_image = distorted_image.resize((28, 28))
                rotated_image = np.array(restored_image)
        
            # shift up, down, left, right
            shift_up = shift_image(rotated_image, direction='up')
            shift_down = shift_image(rotated_image, direction='down')
            shift_left = shift_image(rotated_image, direction='left')
            shift_right = shift_image(rotated_image, direction='right')
            # save to it appropriate container
            deg_container['up'].append(shift_up)
            deg_container['down'].append(shift_down)
            deg_container['left'].append(shift_left)
            deg_container['right'].append(shift_right)
    print('hey')
    for dict_item in container:
        for key, value in dict_item.items():
            dict_item[key] = np.array(value)
    
    print('Setup Complete ', count)
    return container
print('setup complete')

In [ ]:
container = create_rotated_shifted_images(X_train)

In [ ]:
len(container)

In [ ]:
def augment_data(container, dataset, target):
    directions = ['left', 'right', 'up', 'down']
    augment_target = target.copy()
    remove_original = True
    for index, item in enumerate(container):
        for bearing in directions:
            images = item[bearing]
            if remove_original:
                dataset = np.r_[dataset, images]
                target = np.r_[target, augment_target]
            else:
                dataset = images
                remove_original = True
            
        print(dataset.shape, target.shape)
        # quicken garbarge collections, because python is like justice, 
        # slow in catching criminals (unwanted data)
    del directions, container
    return dataset, target
                      

In [ ]:
X_train_aug, y_train_aug = augment_data(container, X_train.values, y_train.values)

In [ ]:
magic_no = 69333
plt.imshow(X_train_aug[magic_no].reshape(28, 28))
print(y_train_aug[magic_no])

In [ ]:
# check the frequency of each image number to ensure that
# it did not deviate from it original 
x, y = np.unique(y_train_aug, return_counts=True)
plt.bar(x, y)

In [ ]:
# shuffle the data
shuffled_indices = np.random.permutation(len(X_train_aug))
X_train_aug = X_train_aug[shuffled_indices]
y_train_aug = y_train_aug[shuffled_indices]

X_train_aug.shape, y_train_aug.shape

In [ ]:
magic_no = 340096
plt.imshow(X_train_aug[magic_no].reshape(28, 28))
print(y_train_aug[magic_no])

In [ ]:
X_train_2, X_val, y_train_2, y_val = train_test_split(X_train_aug, y_train_aug, random_state=8)
X_train_2.shape, X_val.shape, y_train_2.shape, y_val.shape

In [ ]:
log_reg = LogisticRegression()
log_reg.fit(X_train_2, y_train_2)
print('Training score: ', log_reg.score(X_train_2, y_train_2))
print('Test score: ', log_reg.score(X_val, y_val))
print('Ambitious: ', log_reg.score(X_train, y_train))
print('Overly Ambitious', log_reg.score(X_test, y_test))

In [ ]:
predictions = cross_val_predict(log_reg, X_train_2, y_train_2, cv=2)

In [ ]:
conf_mx = confusion_matrix(y_train_2, predictions)
row_sum = conf_mx.sum(axis=1, keepdims=True)
norm_conf_mx = conf_mx / row_sum
np.fill_diagonal(norm_conf_mx, 0)
plt.matshow(norm_conf_mx, cmap=plt.cm.gray)
plt.xlabel('Predicted Class')
plt.ylabel('True Class')

In [ ]:
print(conf_mx)